<a href="https://colab.research.google.com/github/jiho-kang/NLP_RecSys_Project/blob/main/KoBERT_tagging_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab 환경에서 보는 것을 권장합니다.

##### feature에서 unique한 카테고리와 빈도 가져오는 함수

In [ ]:
def get_category(df, columns):
  data = df[columns]
  category = {}
  multi = False
  for word in data:
    lst = []
    if word == None:
      word = 'None'
    
    try:
      if ',' in word:
        lst = [x.strip() for x in word.split(',')]
      else:
        lst.append(word)
      for i in lst:
        if i in category:
          category[i] += 1
        else:
          category[i] = 1
    except:
      print(word)
    if ',' in word:
      multi = True

  category = list(sorted(category.items(), key=lambda item: item[1], reverse=True))
  if multi == True:
    category.append('중복 있음')
  else:
    category.append('중복 없음')
  return category

### 환경설정, 파일 불러오기

In [ ]:
# ! pip install openpyxl==3.0.0
# ! pip install konlpy
# !pip install mxnet-cu101
# !pip install gluonnlp pandas tqdm
# !pip install sentencepiece==0.1.85
# !pip install transformers==2.1.1
# !pip install torch==1.3.1
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import time
import numpy as np
import re
import pickle
from sklearn.preprocessing import MinMaxScaler
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
from transformers import BertModel, BertTokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import time

# 모델
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook


from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
drive.mount('/content/drive')
policy_label = pd.DataFrame(pd.read_pickle('/content/drive/MyDrive/웰로/data/our_final0117.pkl'))
user = pd.DataFrame(pd.read_pickle('/content/drive/MyDrive/웰로/data/user_final0117.pkl'))

policy = pd.read_excel('/content/drive/MyDrive/웰로/data/policy_metetable(AIB전달용).xlsx')
policy = policy.where(pd.notnull(policy), None) # Nan to None
policy = policy[['정책ID', '정책서비스ID', '서비스명', '소관기관', '소관기관유형.1', '생애주기', '신청절차', '선정기준', '지원유형', '서비스목적', '지원내용', '지원대상']]

Mounted at /content/drive


In [ ]:
policy_label.columns

Index(['정책ID', '정책서비스ID', '서비스명', '소관기관', '소관기관유형.1', '생애주기', '정책신청절차', '선정기준',
       '정책지원유형', '서비스목적', '지원내용', '지원대상', '대상연령시작', '대상연령끝', '시도', '시군구',
       '중위소득', '관심상황특성', '대상특성', '대상특성상세', 'MB_15', '소관기관유형', '지원유형', '지원유형상세',
       '신청절차', '성별', '결혼', '학력', '가구원', '자녀', '자녀상세', '직장', '직장상세'],
      dtype='object')

In [ ]:
policy.columns

Index(['정책ID', '정책서비스ID', '서비스명', '소관기관', '소관기관유형.1', '생애주기', '신청절차', '선정기준',
       '지원유형', '서비스목적', '지원내용', '지원대상'],
      dtype='object')

In [ ]:
policy_label.head()

In [ ]:
policy.head()

In [ ]:
for name in ['시도', '시군구', '관심상황특성', '대상특성','MB_15','지원유형', '성별', '결혼', '학력', '가구원', '자녀', '직장']:
  print(f'============================{name}================================')
  print(get_category(policy_label, name))
  print()

============================시도================================
[('경기도', 10221), ('경상북도', 9375), ('경상남도', 8430), ('전라남도', 8005), ('강원도', 6899), ('서울특별시', 6473), ('충청남도', 5951), ('전국', 5616), ('전라북도', 5139), ('부산광역시', 4951), ('충청북도', 4885), ('인천광역시', 3035), ('대구광역시', 2952), ('대전광역시', 2158), ('광주광역시', 2031), ('울산광역시', 2022), ('제주특별자치도', 519), ('세종특별자치시', 315), '중복 있음']

============================시군구================================
[('전체', 6059), ('None', 5616), ('동구', 1771), ('중구', 1724), ('서구', 1584), ('', 1457), ('북구', 1266), ('남구', 1205), ('고성군', 744), ('창원시', 521), ('제주시', 519), ('강서구', 511), ('충주시', 504), ('익산시', 499), ('함양군', 492), ('합천군', 488), ('김해시', 474), ('장성군', 472), ('포항시', 469), ('수원시', 463), ('여수시', 462), ('거창군', 460), ('양산시', 460), ('광양시', 456), ('하동군', 455), ('나주시', 447), ('구미시', 442), ('화성시', 440), ('진안군', 438), ('예산군', 435), ('밀양시', 431), ('안동시', 430), ('성주군', 429), ('목포시', 429), ('의령군', 428), ('영광군', 425), ('울진군', 421), ('논산시', 421), ('함안군', 420), ('제천시', 420), ('순천시

In [ ]:
for name in ['시도', '시군구', '관심상황특성', '대상특성','MB_15','지원유형', '성별', '결혼', '학력', '가구원', '자녀', '직장']:
  print(f'============================{name}================================')
  label_count = policy_label[name].value_counts()
  a = 0
  b = 0
  for label in label_count.keys():
    if ',' in label:
      print(label, label_count[label])
      a += label_count[label]
    
    # if sum(label.count('무관|해당사항없음|None| ')) >:
      

  print(f'중복인 데이터는 전체 {round(a / len(policy_label) * 100, 2)}%를 차지')
  print()


============================시도================================
서울특별시,부산광역시,대구광역시,광주광역시,울산광역시 52
부산광역시,대구광역시,인천광역시 39
부산광역시,대구광역시,광주광역시,울산광역시 19
서울특별시,부산광역시,대구광역시,인천광역시,대전광역시,울산광역시 17
서울특별시,부산광역시,대구광역시,인천광역시,광주광역시,대전광역시 12
서울특별시,부산광역시,대구광역시,인천광역시 1
중복인 데이터는 전체 0.16%를 차지

============================시군구================================
여수시,광양시 27
서구,서구,서구 24
강남구,남구,남구,남구,남구 19
남구,남구,남구,남구 19
중구,중구,중구,중구,중구,중구 17
성북구,북구,북구,북구,북구 17
강북구,북구,북구,북구,북구 16
동구,동구,동구,남동구 15
강서구,서구,강서구,서구,서구,서구,서구 12
남양주시,양주시 10
원주시,강릉시 1
중구,중구,중구,중구 1
중복인 데이터는 전체 0.2%를 차지

============================관심상황특성================================
개인교육지원, 기업금융지원, 기업교육지원 1388
개인교육지원, 결혼예정 438
기업금융지원, 기업교육지원 318
개인교육지원, 기업금융지원 257
개인교육지원, 폭력/범죄/중독지원 201
기업금융지원, 자동차/교통지원 143
개인교육지원, 자동차/교통지원 79
기업금융지원, 은퇴예정 50
개인금융지원, 개인교육지원 26
개인금융지원, 기업금융지원 19
기업금융지원, 폭력/범죄/중독지원 7
개인교육지원, 기업금융지원, 기업교육지원, 자동차/교통지원 6
개인교육지원, 기업금융지원, 자동차/교통지원 5
개인교육지원, 입대예정 4
기업금융지원, 기업교육지원, 자동차/교통지원 4
개인교육지원, 기업금융지원, 결혼예정 3
폭력/범죄/중독지원, 자동차/교통지원 2
개인교육지원, 은퇴예정

In [ ]:
# # 대상특성에 중복포함된건 일단 제외해서 데이터 만듬
# policy_y = policy_label[~policy_label.대상특성.str.contains(',')]
# policy_x = pd.merge(policy, policy_y[['정책ID']], on='정책ID', how='inner')

# policy_x.sort_values('정책ID', inplace=True)
# policy_y.sort_values('정책ID', inplace=True)

# print(len(policy_x), len(policy_y))
# print()

# print(policy_x.columns)
# print(policy_y.columns)
# print()

# # 같은 값들을 가지고 있음
# list(policy_x.정책ID) == list(policy_y.정책ID)

In [ ]:
# # random 하게 1만개의 정책만 가져옴
# policy_x = policy_x.sample(20000, random_state=2022)
# policy_x = pd.merge(policy_y, policy_x[['정책ID']], on='정책ID', how='inner')

# policy_x.sort_values('정책ID', inplace=True)
# policy_y.sort_values('정책ID', inplace=True)

# print(len(policy_x), len(policy_y))
# print()

# print(policy_x.columns)
# print(policy_y.columns)
# print()

# # 같은 값들을 가지고 있음
# list(policy_x.정책ID) == list(policy_y.정책ID)

In [ ]:
policy_label.columns

Index(['정책ID', '정책서비스ID', '서비스명', '소관기관', '소관기관유형.1', '생애주기', '정책신청절차', '선정기준',
       '정책지원유형', '서비스목적', '지원내용', '지원대상', '대상연령시작', '대상연령끝', '시도', '시군구',
       '중위소득', '관심상황특성', '대상특성', '대상특성상세', 'MB_15', '소관기관유형', '지원유형', '지원유형상세',
       '신청절차', '성별', '결혼', '학력', '가구원', '자녀', '자녀상세', '직장', '직장상세'],
      dtype='object')

# 데이터 준비 모델
특수기호 제거, 키워드 치환, 라벨값 숫자로 변환

In [ ]:
# 한글, 숫자, 영어 빼고 전부 제거
def sub_special(s):
  return re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z ~]',' ',str(s))

# 형태소 분석, 불용어 처리
def morph_and_stopword(s):
  STOP_WORDS = ['의','가','은','들','는','좀','잘','걍','과','를','으로','에','와','한','하다']
  okt = Okt()
  token_ls = []

  #형태소 분석
  tmp = okt.morphs(s, stem=True)

  #불용어 처리
  for token in tmp:
    if token not in STOP_WORDS:
      token_ls.append(token)
  return ' '.join(token_ls)

In [ ]:
def change_word_list(feature):
  target_word = []
  target_tag = []

  if feature == '대상특성':
    target_word.append('농업|농축|어업|원예|축산|임업|농산물|재배|귀농|영농|축산|과수|과일|유기농|화훼|원예|작물|토지|토양|농지|농가|소유|연근해|구획|어선|수산|양식|임업|사육')
    target_tag.append('농축수산인')

    target_word.append('환자|입원|질병|부상|격리|난치|암|치매|요양')
    target_tag.append('질병/부상/질환자')

    target_word.append('임신|임산부|출산|산모')
    target_tag.append('임신부')

    target_word.append('영유아|영아|태아|신생아|모유|분만')
    target_tag.append('출산후')

    target_word.append('유공자|독립|보훈|순국선열|애국지사')
    target_tag.append('국가유공자')
  
  elif feature == '관심상황특성':    
    target_word.append('생활|생활')
    target_tag.append('개인')

    target_word.append('경영|창업|영세|자영업|소상공인|사업')
    target_tag.append('기업')

    target_word.append('자금')
    target_tag.append('금융')
  
  elif feature == '신청절차':
    target_word.append('방문|우편|서면|팩스|은행|영업|사무소|내방|동사무소')
    target_tag.append('오프라인')

    target_word.append('모바일|ARS|전화|민원24|이메일|인터넷|홈택스|홈페이지|앱|웹')
    target_tag.append('온라인')

  elif feature == '성별':
    target_word.append('군인|의경|입대|장병|전립선|남자')
    target_tag.append('남성')

    target_word.append('출산|임신|임산부|여자|산모')
    target_tag.append('여성')

  elif feature == '결혼':
    target_word.append('가정|아동|다문화|가족|자녀')
    target_tag.append('기혼')
  
  elif feature == '자녀':
    target_word.append('영아|유아|영유아|유치원|아동|어린이|초중고등|대학')
    target_tag.append('있음')
  
  elif feature == '직장':
    target_word.append('일자리|취업|구직|실업')
    target_tag.append('구직자/실업자')

    target_word.append('종사자|직원|직장')
    target_tag.append('근로자/직장인')
  
  return target_word, target_tag


In [ ]:
def change_keyword(df_x, df_y, feature, num):
  df_x = df_x.sort_values('정책ID').fillna('')
  df_y = df_y.sort_values('정책ID').fillna('')
  cols = df_x.columns[2:]
  # feature에 따라 바꿀 단어와 태그를 가져옴
  target_word, target_tag = change_word_list(feature)
  
  # x_train 만들기: 열마다 단어를 바꿈
  for col in cols:
    if col == '생애주기':
      df_x[col] = df_x[col].apply(sub_special)
      continue

    for word, tag in zip(target_word, target_tag):
      # 불용어 처리 후 형태소 분석
      # df_x[col] = df_x[col].apply(sub_special).apply(morph_and_stopword).apply(lambda x: re.sub(word, tag, x))
      
      # 특수기호만 제거
      df_x[col] = df_x[col].apply(sub_special).apply(lambda x: re.sub(word, tag, x))
  
  # 데이터를 num개만 랜덤하게 가져옴
  df_y = df_y[['정책ID',feature]].sample(num, random_state=2022)
  df_x = df_x.sample(num, random_state=2022)

  # label의 고유 카테고리 가져오기
  label_lst = []
  for data in get_category(df_y, feature)[:-1]:
    label_lst.append(data[0])
  print('label의 카테고리는:', label_lst)
  
  # y_train 만들기
  y_train = []

  for label in df_y[feature]:
    labels = [str(label_lst.index(data.strip())) for data in label.split(',')]
    y_train.append(','.join(labels))
  df_y['라벨변환'] = y_train

  print(f'change_keyword 함수에서 Input과 Output은 올바르게 매칭되어 있는가? {list(df_x.정책ID) == list(df_y.정책ID)}')

  return df_x, df_y, label_lst

In [ ]:
def pytorch_data(df_x, df_y, feature, num):
  df_x, df_y, label_lst = change_keyword(df_x, df_y, feature, num)
  x_train = []
  for i in range(len(df_x)):
    x_train.append(' '.join(df_x.iloc[i].values[2:]))
  
  df = df_y[['정책ID', '라벨변환']]
  df['데이터'] = x_train

  # data = [[a, str(b)] for a, b in zip(x_train, y_train)]
  # train, val = train_test_split(data, train_size = 0.7, random_state=2022, shuffle=True)

  train, val = train_test_split(df, train_size = 0.7, random_state=2022, shuffle=True)
  return train, val, label_lst

In [ ]:
# train, val, category = pytorch_data(policy, policy_label, '성별', 10000)

In [ ]:
# train.head()

In [ ]:
# torch_train = train[['데이터', '라벨변환']].values
# torch_val = val[['데이터', '라벨변환']].values

# torch_train[0]

In [ ]:
# print(len(torch_train), len(torch_val))

# torch 모델

In [ ]:
def torch1(trian, val, category):

  class BERTDataset(Dataset):
      def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                  pad, pair):
          transform = nlp.data.BERTSentenceTransform(
              bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

          self.sentences = [transform([i[sent_idx]]) for i in dataset]
          self.labels = [np.int32(i[label_idx]) for i in dataset]

      def __getitem__(self, i):
          return (self.sentences[i] + (self.labels[i], ))

      def __len__(self):
          return (len(self.labels))


  class BERTClassifier(nn.Module):
      def __init__(self,
                  bert,
                  hidden_size = 768,
                  num_classes=2,
                  dr_rate=None,
                  params=None):
          super(BERTClassifier, self).__init__()
          self.bert = bert
          self.dr_rate = dr_rate
                  
          self.classifier = nn.Linear(hidden_size , num_classes)
          if dr_rate:
              self.dropout = nn.Dropout(p=dr_rate)
      
      def gen_attention_mask(self, token_ids, valid_length):
          attention_mask = torch.zeros_like(token_ids)
          for i, v in enumerate(valid_length):
              attention_mask[i][:v] = 1
          return attention_mask.float()

      def forward(self, token_ids, valid_length, segment_ids):
          attention_mask = self.gen_attention_mask(token_ids, valid_length)
          
          _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict=False)

          if self.dr_rate:
              out = self.dropout(pooler)
          return self.classifier(out) 


  def calc_accuracy(X,Y):
      max_vals, max_indices = torch.max(X, 1)
      train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
      
      return train_acc


  ## 데이터 변형
  torch_train = train[['데이터', '라벨변환']].values
  torch_val = val[['데이터', '라벨변환']].values

  # device 
  device = torch.device("cuda:0")
  bertmodel, vocab = get_pytorch_kobert_model()

  # tokenizer
  tokenizer = get_tokenizer()
  tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

  # parameter
  max_len = 64
  batch_size = 50
  warmup_ratio = 0.1
  num_epochs = 7
  max_grad_norm = 1
  log_interval = 200
  learning_rate =  0.005
  num_classes = len(category)


  data_train = BERTDataset(torch_train, 0, 1, tok, max_len, True, False)
  data_test = BERTDataset(torch_val, 0, 1, tok, max_len, True, False)

  train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5, shuffle=True)
  test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5, shuffle=True)


  model = BERTClassifier(bertmodel, num_classes=num_classes, dr_rate=0.5).to(device)

  no_decay = ['bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
      {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
      {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
  ]

  optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
  loss_fn = nn.CrossEntropyLoss()


  t_total = len(train_dataloader) * num_epochs
  warmup_step = int(t_total * warmup_ratio)

  scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


  for e in range(num_epochs):
      train_acc = 0.0
      test_acc = 0.0
      model.train()
      for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
          optimizer.zero_grad()
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)
          valid_length= valid_length
          label = label.long().to(device)
          out = model(token_ids, valid_length, segment_ids)
          loss = loss_fn(out, label)
          loss.backward()
          torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
          optimizer.step()
          scheduler.step()
          train_acc += calc_accuracy(out, label)
      print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
      model.eval()
      for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)
          valid_length= valid_length
          label = label.long().to(device)
          out = model(token_ids, valid_length, segment_ids)
          test_acc += calc_accuracy(out, label)
      print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
  
  return model

In [ ]:
def softmax(vals, idx):
    valscpu = vals.cpu().detach().squeeze(0)
    a = 0
    for i in valscpu:
        a += np.exp(i)
    return ((np.exp(valscpu[idx]))/a).item() * 100

def testModel(model, name, category, seq):
    # device 
    device = torch.device("cuda:0")
    bertmodel, vocab = get_pytorch_kobert_model()

    # tokenizer
    tokenizer = get_tokenizer()
    tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

    cate = category
    tmp = [seq]
    transform = nlp.data.BERTSentenceTransform(tok, 64, pad=True, pair=False)
    tokenized = transform(tmp)

    model.eval()
    result = model(torch.tensor([tokenized[0]]).to(device), [tokenized[1]], torch.tensor(tokenized[2]).to(device))
    idx = result.argmax().cpu().item()
    print(f"정책의 주된 {name} 태그는:", cate[idx])
    print("신뢰도는:", "{:.2f}%".format(softmax(result,idx)))

# torch 실행 창

In [ ]:
for name in policy_label.columns[2:]:
  if policy_label[name].dtype == 'int64' :
    continue
  if '중복 없음' in get_category(policy_label, name):
    print(name)
    print(get_category(policy_label, name))
    print()

소관기관
[('경상남도', 598), ('충청북도 충주시', 504), ('경상남도 창원시', 502), ('전라북도 익산시', 499), ('경상남도 함양군', 492), ('경상남도 합천군', 488), ('충청남도', 486), ('제주특별자치도', 476), ('전라남도 장성군', 472), ('경상남도 김해시', 470), ('대구광역시', 470), ('강원도', 464), ('경상남도 거창군', 460), ('경상북도 포항시', 459), ('경상남도 양산시', 456), ('경상남도 하동군', 455), ('전라남도 나주시', 447), ('전라북도 진안군', 438), ('전라남도 여수시', 435), ('충청남도 예산군', 435), ('경상남도 밀양시', 431), ('경상북도 성주군', 429), ('전라남도 광양시', 429), ('전라남도 목포시', 429), ('경상남도 의령군', 428), ('경기도 화성시', 427), ('경상북도 구미시', 426), ('경기도 수원시', 425), ('울산광역시', 425), ('전라남도 영광군', 425), ('경상북도 안동시', 422), ('경상북도 울진군', 421), ('경상북도', 421), ('충청남도 논산시', 421), ('충청북도 제천시', 420), ('전라남도 순천시', 417), ('강원도 원주시', 414), ('경상남도 진주시', 413), ('강원도 화천군', 412), ('충청북도 보은군', 412), ('경상북도 예천군', 410), ('경상남도 함안군', 409), ('충청북도', 409), ('충청북도 영동군', 409), ('경상북도 청송군', 404), ('충청북도 괴산군', 404), ('강원도 횡성군', 400), ('경기도 연천군', 398), ('경상북도 경주시', 398), ('충청북도 증평군', 398), ('충청남도 홍성군', 397), ('경상남도 고성군', 394), ('농림축산식품부', 392), ('경상남도 사천시', 390), ('충

## 신청절차 KoBERT

In [ ]:
train, val, category = pytorch_data(policy, policy_label, '신청절차', 10000)

label의 카테고리는: ['무관', '오프라인', '온라인']
change_keyword 함수에서 Input과 Output은 올바르게 매칭되어 있는가? True


In [ ]:
train

,정책ID,라벨변환,데이터
79351,79352,0,인플루엔자 예방접종 지원 충청북도 단양군 시군구 영 유아 노년기 의료지원 정...
10290,10289,0,시간 연장 어린이집 운영 지원 경기도 가평군 시군구 청년 중장년 노년기 시간 ...
43999,43999,1,가스안전 관리자금 융자 부산광역시 수영구 시군구 청년 중장년 노년기 오프라인 소...
71669,71670,1,독립유공자 지원수당 충청남도 논산시 시군구 영 유아 아동 청소년 청년 중장년 ...
36453,36452,0,만 3~5세 유치원 유아 학비 지원 광주광역시 서구 시군구 영 유아 오프라인 ...
...,...,...,...
33333,33332,0,농어촌 장애인주택 주거용 편의시설 설치 지원 경상북도 문경시 시군구 영 유아 아동...
40971,40970,1,기초생활수급자 장제급여 지원 대전광역시 서구 시군구 청년 중장년 노년기 읍 면 ...
58911,58912,1,저소득층 자산형성 지원 희망 내일키움통장 사업 전라남도 광역시도 청년 중장년 ...
79654,79655,1,장애인 보조기기 지원 충청북도 증평군 시군구 아동 청소년 청년 중장년 노년기 ...


In [ ]:
model_1 = torch1(train, val, '신청절차')

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:109: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/140 [00:00<?, ?it/s]

epoch 1 train acc 0.5501428571428575


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:124: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 6 test acc 0.6303333333333333


  0%|          | 0/140 [00:00<?, ?it/s]

epoch 7 train acc 0.6272857142857142


  0%|          | 0/60 [00:00<?, ?it/s]

epoch 7 test acc 0.6303333333333333


In [ ]:
get_category(policy_label, '신청절차')

[('무관', 54725), ('오프라인', 29338), ('온라인', 4423), '중복 없음']

In [ ]:
testModel(model_1, '신청절차', category, "근처 방문 동사무소 방문 방문신청하세요 .")

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
정책의 주된 신청절차 태그는: 무관
신뢰도는: 63.19%


In [ ]:
testModel(model_1, '신청절차', category, "인터넷, 홈택스, 앱, 홈페이지에서 온라인 온라인신청하세요 .")

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
정책의 주된 신청절차 태그는: 무관
신뢰도는: 63.19%


In [ ]:
train, val, category = pytorch_data(policy, policy_label, '성별', 10000)

label의 카테고리는: ['무관', '여성', '남성']
change_keyword 함수에서 Input과 Output은 올바르게 매칭되어 있는가? True


In [ ]:
train

,정책ID,라벨변환,데이터
79351,79352,1,인플루엔자 예방접종 지원 충청북도 단양군 시군구 영 유아 노년기 의료지원 정...
10290,10289,0,시간 연장 어린이집 운영 지원 경기도 가평군 시군구 청년 중장년 노년기 시간 ...
43999,43999,0,가스안전 관리자금 융자 부산광역시 수영구 시군구 청년 중장년 노년기 방문 소상공...
71669,71670,2,독립유공자 지원수당 충청남도 논산시 시군구 영 유아 아동 청소년 청년 중장년 ...
36453,36452,0,만 3~5세 유치원 유아 학비 지원 광주광역시 서구 시군구 영 유아 방문 동...
...,...,...,...
33333,33332,0,농어촌 장애인주택 주거용 편의시설 설치 지원 경상북도 문경시 시군구 영 유아 아동...
40971,40970,2,기초생활수급자 장제급여 지원 대전광역시 서구 시군구 청년 중장년 노년기 읍 면 ...
58911,58912,0,저소득층 자산형성 지원 희망 내일키움통장 사업 전라남도 광역시도 청년 중장년 ...
79654,79655,0,장애인 보조기기 지원 충청북도 증평군 시군구 아동 청소년 청년 중장년 노년기 ...


In [ ]:
model_2 = torch1(train, val, '성별')

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:109: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/140 [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:
get_category(policy_label, '성별')

In [ ]:
testModel(model_1, '신청절차', category, "근처 방문 동사무소 방문 방문신청하세요 .")

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
정책의 주된 신청절차 태그는: 무관
신뢰도는: 63.19%


In [ ]:
testModel(model_1, '신청절차', category, "인터넷, 홈택스, 앱, 홈페이지에서 온라인 온라인신청하세요 .")

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
정책의 주된 신청절차 태그는: 무관
신뢰도는: 63.19%


# torch 모델 (쪼개기)

In [ ]:
torch_train = train[['데이터', '라벨변환']].values
torch_val = val[['데이터', '라벨변환']].values

torch_train[0]

array(['인플루엔자 예방접종 지원 충청북도 단양군 시군구 영 유아  노년기   의료지원  정보제공 예방접종률 향상과 질병 부담 감소 65세 이상 노인  생후 6개월~만 12세 이하 아동 및 임신부에게 인플루엔자 백신 매년 10월 시작 예정  접종    건강 의료 사망 예방 검사 예방접종  건강 의료 사망 의료지원 노인  건강 의료 사망 의료지원 일반 65세 이상 노인 및 생후 6개월~12세 이하 아동  임신부  일반',
       '0'], dtype=object)

In [ ]:
torch_val

array([['과원 농작업으로 지원 경상남도 함안군 시군구 청년  중장년  노년기 시군농업기술센터   시군은 자체심의위원회를 구성하여 기업계획 기업비 산출근거 의 적정성  운영실태 등을 종합평가하여 기업대상자 선정      5년간 지원내용 확인 후 동일기업 기 지원자 신청 시 뒷순위로 심사    과수농가의 과원관리 효율성을 높여 안정적 생산기반 구축   수확  예초 작업  농약 살포 등에 드는 인건비 등의 절감 효과 작업로가 없는 과원 중심으로 작업으로 확장 포장 지원    기업 기업 업종별민원 농업 축산업 어업   지원대상   1ha 이상 과수 재배농가 수출농가  APC공동출하농가 등       농업인   농어업 농어촌 및 식품산업 기본법  제3조세2호  농축수산인',
        '2'],
       ['진로진학 프로그램 운영 대전광역시 유성구 시군구 아동 청소년 온라인 없음 상담 법률지원  교육 용역 진로진학 프로그램 운영 대학입시설명회 대입전략특강  및 1 1진학상담    결혼 육아 교육 중학교 고등학교 진로와 입시  결혼 육아 교육 중학교 고등학교 상담 선도 관내 초 중 고등학생 및 학부모  학생',
        '0'],
       ['초정행궁 사용료 감면 30   청주시시설관리공단 공공기관 등 영 유아  아동 청소년  청년  중장년  노년기   문화 여가지원  감면 면제 사용료 감면   사용료 감면 30     단 비수기에만 감면 적용  7 8월 토 일요일  공휴일 제외   20200623~20300630  기타 시설사용료 감면 대상임을 증명할 수 있는 증빙서류  신분증 또는 등본 지참   여가 문화 출입국 관광 여행 관광  여가 문화 출입국 관광 여행 여행   주민등록법에 따라 청주시에 주소를 둔 사람   막내가 18세 이하인 2자녀 이상 가정   장애인 복지법에 따라 등록된 장애인   기초개인수급자  장애인  저소득층  다자녀 가정  일반',
        '0'],
       ...,
       ['국민임대주택 공급 전라북도 광역

In [ ]:
[np.int32(i[1]) for i in torch_val]

['2',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '2',
 '0',
 '0',
 '0',
 '0',
 '0',
 '2',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '2',
 '1,6',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '2',
 '0',
 '2',
 '0',
 '2',
 '0',
 '0',
 '0',
 '0',
 '5',
 '0',
 '0',
 '1',
 '2',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '2',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '5',
 '0',
 '0',
 '1',
 '0',
 '2',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1,2,3',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '2',
 '0',
 '0',
 '0',
 '5',
 '0',
 '1,6',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '2',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '2',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '2',
 '2,4',
 '0',
 '2',
 '0',
 '0',
 '0',
 '0',
 '0',
 '2',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '2',
 '0',
 '1',
 '1',


In [ ]:
# print(len(torch_train), len(torch_val))

7000 3000


In [ ]:
device = torch.device("cuda:0")
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
a = max(len(sent[0]) for sent in torch_train)
b = np.mean([len(sent[0]) for sent in torch_train], dtype=int)
print(a, b)
train_maxlen = (a + b) /2
print(train_maxlen)

4295 614
2454.5


In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
max_len = 64
batch_size = 1
warmup_ratio = 0.1
num_epochs = 7
max_grad_norm = 1
log_interval = 200
learning_rate =  0.005

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [i[label_idx] for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
data_train = BERTDataset(torch_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(torch_val, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5, shuffle=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict=False)

        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out) 

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

loss_fn = nn.CrossEntropyLoss()



t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length

        out = model(token_ids, valid_length, segment_ids)

        label = label[0]
        print(label)
        
        if ',' in label:          
            labels = list(map(int, label.split(',')))
            loss = 0
            for i in range(len(labels)):
              label = labels[i]
              # label = tuple(label, )
              label = torch.from_numpy(np.array([np.int32(label)])).long().to(device)
              loss += loss_fn(out, label)
              loss /= len(labels)

        else:
          label = torch.from_numpy(np.array([np.int32(label)])).long().to(device)
          loss = loss_fn(out, label)
        
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        train_acc += calc_accuracy(out, label)
        # print(train_acc)
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/7000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


0


RuntimeError: ignored

In [ ]:
def softmax(vals, idx):
    valscpu = vals.cpu().detach().squeeze(0)
    a = 0
    for i in valscpu:
        a += np.exp(i)
    return ((np.exp(valscpu[idx]))/a).item() * 100

def testModel(model, seq):
    cate = category
    tmp = [seq]
    transform = nlp.data.BERTSentenceTransform(tok, max_len, pad=True, pair=False)
    tokenized = transform(tmp)

    model.eval()
    result = model(torch.tensor([tokenized[0]]).to(device), [tokenized[1]], torch.tensor(tokenized[2]).to(device))
    idx = result.argmax().cpu().item()
    print("정책의 [대상특성] 태그는:", cate[idx])
    print("신뢰도는:", "{:.2f}%".format(softmax(result,idx)))

In [ ]:
testModel(model, "임산부가 어쩌고 여성이 저쩌고 남성도 여성 여성 여성 여성")

NameError: ignored

----


# -----------여기부터 연습코드---------

# pytorch 1 - 대상특성

### 환경설정

In [ ]:
# !pip install mxnet-cu101
# !pip install gluonnlp pandas tqdm
# !pip install sentencepiece==0.1.85
# !pip install transformers==2.1.1
# !pip install torch==1.3.1
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook


from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

## 1 전처리 없는 데이터

### 데이터 분석 및 준비

In [ ]:
policy_x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 33 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   정책ID      10000 non-null  int64 
 1   정책서비스ID   10000 non-null  object
 2   서비스명      10000 non-null  object
 3   소관기관      10000 non-null  object
 4   소관기관유형.1  10000 non-null  object
 5   생애주기      9881 non-null   object
 6   정책신청절차    8023 non-null   object
 7   선정기준      5749 non-null   object
 8   정책지원유형    9326 non-null   object
 9   서비스목적     10000 non-null  object
 10  지원내용      10000 non-null  object
 11  지원대상      10000 non-null  object
 12  대상연령시작    10000 non-null  int64 
 13  대상연령끝     10000 non-null  int64 
 14  시도        10000 non-null  object
 15  시군구       9267 non-null   object
 16  중위소득      903 non-null    object
 17  관심상황특성    10000 non-null  object
 18  대상특성      10000 non-null  object
 19  대상특성상세    10000 non-null  object
 20  MB_15     10000 non-null  object
 21  소관기관유형    100

In [ ]:
policy_y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72396 entries, 0 to 88484
Data columns (total 33 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   정책ID      72396 non-null  int64 
 1   정책서비스ID   72396 non-null  object
 2   서비스명      72396 non-null  object
 3   소관기관      72396 non-null  object
 4   소관기관유형.1  72396 non-null  object
 5   생애주기      71471 non-null  object
 6   정책신청절차    57704 non-null  object
 7   선정기준      41263 non-null  object
 8   정책지원유형    67562 non-null  object
 9   서비스목적     72396 non-null  object
 10  지원내용      72396 non-null  object
 11  지원대상      72396 non-null  object
 12  대상연령시작    72396 non-null  int64 
 13  대상연령끝     72396 non-null  int64 
 14  시도        72396 non-null  object
 15  시군구       67079 non-null  object
 16  중위소득      6159 non-null   object
 17  관심상황특성    72396 non-null  object
 18  대상특성      72396 non-null  object
 19  대상특성상세    72396 non-null  object
 20  MB_15     72396 non-null  object
 21  소관기관유형    72

In [ ]:
category = ['해당사항없음', '장애인', '농축수산인', '질병/부상/질환자', '난임', '임신부', '출산후', '국가유공자', '제대군인', '다문화가족', '북한이탈주민', '한부모가정/조손가정', '1인가구', '다자녀가정', '독거노인', '입양가정', '외국인/재외국인', '한센인', '이산가족', '기관/시설']
category.index('장애인')

1

In [ ]:
def torch_dataset_to_modeldata(x_df, y_df):
  x_train = x_df.fillna('')
  x_train = x_train['서비스명'] + '.\n' + x_train['선정기준'] + '.\n' + x_train['서비스목적'] + '.\n' + x_train['지원내용'] + '.\n' + x_train['지원대상']
  # x_train = x_train['서비스명'] + '.\n' + x_train['지원대상']
  x_train = x_train.to_list()

  y = y_df.fillna('')
  y = y['대상특성']
  category = ['해당사항없음', '장애인', '농축수산인', '질병/부상/질환자', '난임', '임신부', '출산후', '국가유공자', '제대군인', '다문화가족', '북한이탈주민', '한부모가정/조손가정', '1인가구', '다자녀가정', '독거노인', '입양가정', '외국인/재외국인', '한센인', '이산가족', '기관/시설']
  y_train = [category.index(data) for data in y ]

  data = [[a, str(b)] for a, b in zip(x_train, y_train)]

  train, val = train_test_split(data, train_size = 0.7, random_state=2022, shuffle=True)
  return train, val

In [ ]:
torch_train, torch_val = torch_dataset_to_modeldata(policy_x, policy_y)

In [ ]:
torch_train[:2]

[['쓰레기 무단투기 신고 및 신고포상금.\n.\n쓰레기 무단투기자에 대한 과태료 부과 및 무단투기 신고자에 대한 신고포상금 제도 시행으로 무분별한 무단투기 예방에 기여.\n무단투기 신고 후 과태료를 부과하였을 때에는 신고자에게 신고포상금을 지급한다. 다만, 폐기물 관리 업무를 직접 수행하는 공무원, 공익근무요원, 공공 근로자 등은 제외된다.\n\n\n\n금융·세금·법률>상담/법률구제>고충상담||환경·재난>생활환경>폐기물.\n○ 무단투기 신고 : 폐기물 무단투기 일시, 장소, 폐기물의 종류를 신고하는 누구나\n\n○ 신고포상금 : 폐기물 무단투기자의 인적 사항이나 인적 사항을 파악할 수 있는 자료를 가지고 신고하는 자\n\n일반',
  '2'],
 ['장애인생활안정지원.\n.\n장애로 인한 경제활동이 어려운 저소득 장애인의 생활안정을 위한 수당 및 의료지원.\n장애수당추가 : 월 12천원 지원\n부부 장애수당 : 월 35천원지원\n월세거주장애인주거비 : 월 50천원 지원\n중증장애인가구 월동비지원 : 년 131천원 지원\n청각장애아동인공달팽이치료지원 : 수술 6백만원 이하 / 재활치료 년 4백만원 이하 \n장루요루장애인의료비지원 : 년 230천원 이하 \n미취업장애인건강검진지원  : 1회 200천원 \n장애인건강교실운영 : 건강교실 운영 프로그램 참여\n\n20170101~20471231\n\n생활·병역>생활지원>장애인.\n장애수당추가 : 18세 이상 중증장애인으로 기준중위 소득 50% 이하 \n부부장애수당 : 부부가 장애인인 세대주\n월세거주장애인주거비 : 세대주가 장애인인 기준중위 소득 80% 이하인 가구\n중증장애인가구 월동비지원 : 중증장애인 가구 대상으로 기준중위 소득 50% 이하\n청각장애아동인공달팽이치료지원 : 15세 이하의 청각장애아동\n장루요루장애인의료비지원 : 장루,요루 장애인으로서 기준중위 소득 120% 이하\n미취업장애인건강검진지원  : 18세 이상의 미취업 장애인 \n장애인건강교실운영 : 등록장애인 및 장애인가족\n\n장애인',
  '0

### torch 모델

In [ ]:
device = torch.device("cuda:0")
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
print(len(torch_train), len(torch_val))

7000 3000


In [ ]:
torch_train[0]

['쓰레기 무단투기 신고 및 신고포상금.\n.\n쓰레기 무단투기자에 대한 과태료 부과 및 무단투기 신고자에 대한 신고포상금 제도 시행으로 무분별한 무단투기 예방에 기여.\n무단투기 신고 후 과태료를 부과하였을 때에는 신고자에게 신고포상금을 지급한다. 다만, 폐기물 관리 업무를 직접 수행하는 공무원, 공익근무요원, 공공 근로자 등은 제외된다.\n\n\n\n금융·세금·법률>상담/법률구제>고충상담||환경·재난>생활환경>폐기물.\n○ 무단투기 신고 : 폐기물 무단투기 일시, 장소, 폐기물의 종류를 신고하는 누구나\n\n○ 신고포상금 : 폐기물 무단투기자의 인적 사항이나 인적 사항을 파악할 수 있는 자료를 가지고 신고하는 자\n\n일반',
 '2']

In [ ]:
a = max(len(sent[0]) for sent in torch_train)
b = np.mean([len(sent[0]) for sent in torch_train], dtype=int)
print(a, b)
train_maxlen = (a + b) /2
print(train_maxlen)

3160 451
1805.5


In [ ]:
l1 = [len(i[0]) for i in torch_train]
l2 = [len(i[0]) for i in torch_val]
max(max(l1),max(l2))

3160

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
max_len = 64
batch_size = 6
warmup_ratio = 0.1
num_epochs = 7
max_grad_norm = 1
log_interval = 200
learning_rate =  0.001

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
data_train = BERTDataset(torch_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(torch_val, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5, shuffle=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=20,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict=False)

        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out) 

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()


t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        train_acc += calc_accuracy(out, label)
        # print(train_acc)
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/1167 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 train acc 0.4265924021708084


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 6 train acc 0.4607969151670952


  0%|          | 0/500 [00:00<?, ?it/s]

epoch 6 test acc 0.4690000000000005


  0%|          | 0/1167 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
def softmax(vals, idx):
    valscpu = vals.cpu().detach().squeeze(0)
    a = 0
    for i in valscpu:
        a += np.exp(i)
    return ((np.exp(valscpu[idx]))/a).item() * 100

def testModel(model, seq):
    cate = category
    tmp = [seq]
    transform = nlp.data.BERTSentenceTransform(tok, max_len, pad=True, pair=False)
    tokenized = transform(tmp)

    model.eval()
    result = model(torch.tensor([tokenized[0]]).to(device), [tokenized[1]], torch.tensor(tokenized[2]).to(device))
    idx = result.argmax().cpu().item()
    print("정책의 [대상특성] 태그는:", cate[idx])
    print("신뢰도는:", "{:.2f}%".format(softmax(result,idx)))

In [ ]:
testModel(model, "어쩌고 저쩌고 특성 장애를 가진 장애인이며, 5~10세 이상 장애를 가진 장애인")

정책의 [대상특성] 태그는: 해당사항없음
신뢰도는: 99.21%


## 2. 전처리한 데이터

### 데이터 분석 및 준비

In [ ]:
policy_x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 33 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   정책ID      10000 non-null  int64 
 1   정책서비스ID   10000 non-null  object
 2   서비스명      10000 non-null  object
 3   소관기관      10000 non-null  object
 4   소관기관유형.1  10000 non-null  object
 5   생애주기      9881 non-null   object
 6   정책신청절차    8023 non-null   object
 7   선정기준      5749 non-null   object
 8   정책지원유형    9326 non-null   object
 9   서비스목적     10000 non-null  object
 10  지원내용      10000 non-null  object
 11  지원대상      10000 non-null  object
 12  대상연령시작    10000 non-null  int64 
 13  대상연령끝     10000 non-null  int64 
 14  시도        10000 non-null  object
 15  시군구       9267 non-null   object
 16  중위소득      903 non-null    object
 17  관심상황특성    10000 non-null  object
 18  대상특성      10000 non-null  object
 19  대상특성상세    10000 non-null  object
 20  MB_15     10000 non-null  object
 21  소관기관유형    100

In [ ]:
policy_y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72396 entries, 0 to 88484
Data columns (total 33 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   정책ID      72396 non-null  int64 
 1   정책서비스ID   72396 non-null  object
 2   서비스명      72396 non-null  object
 3   소관기관      72396 non-null  object
 4   소관기관유형.1  72396 non-null  object
 5   생애주기      71471 non-null  object
 6   정책신청절차    57704 non-null  object
 7   선정기준      41263 non-null  object
 8   정책지원유형    67562 non-null  object
 9   서비스목적     72396 non-null  object
 10  지원내용      72396 non-null  object
 11  지원대상      72396 non-null  object
 12  대상연령시작    72396 non-null  int64 
 13  대상연령끝     72396 non-null  int64 
 14  시도        72396 non-null  object
 15  시군구       67079 non-null  object
 16  중위소득      6159 non-null   object
 17  관심상황특성    72396 non-null  object
 18  대상특성      72396 non-null  object
 19  대상특성상세    72396 non-null  object
 20  MB_15     72396 non-null  object
 21  소관기관유형    72

In [ ]:
category = ['해당사항없음', '장애인', '농축수산인', '질병/부상/질환자', '난임', '임신부', '출산후', '국가유공자', '제대군인', '다문화가족', '북한이탈주민', '한부모가정/조손가정', '1인가구', '다자녀가정', '독거노인', '입양가정', '외국인/재외국인', '한센인', '이산가족', '기관/시설']
category.index('장애인')

1

In [ ]:
def sub_special(s):
  return re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z ~]',' ',str(s))

In [ ]:
def torch_dataset_to_modeldata(x_df, y_df):
  x_train = x_df.fillna('')
  x_train = x_train['서비스명'] + '.\n' + x_train['선정기준'] + '.\n' + x_train['서비스목적'] + '.\n' + x_train['지원내용'] + '.\n' + x_train['지원대상']
  x_train = x_train.apply(sub_special)
  # x_train = x_train['서비스명'] + '.\n' + x_train['지원대상']
  x_train = x_train.to_list()

  y = y_df.fillna('')
  y = y['대상특성']
  category = ['해당사항없음', '장애인', '농축수산인', '질병/부상/질환자', '난임', '임신부', '출산후', '국가유공자', '제대군인', '다문화가족', '북한이탈주민', '한부모가정/조손가정', '1인가구', '다자녀가정', '독거노인', '입양가정', '외국인/재외국인', '한센인', '이산가족', '기관/시설']
  y_train = [category.index(data) for data in y ]

  data = [[a, str(b)] for a, b in zip(x_train, y_train)]

  train, val = train_test_split(data, train_size = 0.7, random_state=2022, shuffle=True)
  return train, val

In [ ]:
torch_train, torch_val = torch_dataset_to_modeldata(policy_x, policy_y)

In [ ]:
torch_train[:2]

[['쓰레기 무단투기 신고 및 신고포상금    쓰레기 무단투기자에 대한 과태료 부과 및 무단투기 신고자에 대한 신고포상금 제도 시행으로 무분별한 무단투기 예방에 기여  무단투기 신고 후 과태료를 부과하였을 때에는 신고자에게 신고포상금을 지급한다  다만  폐기물 관리 업무를 직접 수행하는 공무원  공익근무요원  공공 근로자 등은 제외된다     금융 세금 법률 상담 법률구제 고충상담  환경 재난 생활환경 폐기물    무단투기 신고   폐기물 무단투기 일시  장소  폐기물의 종류를 신고하는 누구나    신고포상금   폐기물 무단투기자의 인적 사항이나 인적 사항을 파악할 수 있는 자료를 가지고 신고하는 자  일반',
  '2'],
 ['장애인생활안정지원    장애로 인한 경제활동이 어려운 저소득 장애인의 생활안정을 위한 수당 및 의료지원  장애수당추가   월 12천원 지원 부부 장애수당   월 35천원지원 월세거주장애인주거비   월 50천원 지원 중증장애인가구 월동비지원   년 131천원 지원 청각장애아동인공달팽이치료지원   수술 6백만원 이하   재활치료 년 4백만원 이하  장루요루장애인의료비지원   년 230천원 이하  미취업장애인건강검진지원    1회 200천원  장애인건강교실운영   건강교실 운영 프로그램 참여  20170101~20471231  생활 병역 생활지원 장애인  장애수당추가   18세 이상 중증장애인으로 기준중위 소득 50  이하  부부장애수당   부부가 장애인인 세대주 월세거주장애인주거비   세대주가 장애인인 기준중위 소득 80  이하인 가구 중증장애인가구 월동비지원   중증장애인 가구 대상으로 기준중위 소득 50  이하 청각장애아동인공달팽이치료지원   15세 이하의 청각장애아동 장루요루장애인의료비지원   장루 요루 장애인으로서 기준중위 소득 120  이하 미취업장애인건강검진지원    18세 이상의 미취업 장애인  장애인건강교실운영   등록장애인 및 장애인가족  장애인',
  '0']]

In [ ]:
# 이전
# torch_train[:2]

### torch 모델

In [ ]:
device = torch.device("cuda:0")
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
print(len(torch_train), len(torch_val))

7000 3000


In [ ]:
a = max(len(sent[0]) for sent in torch_train)
b = np.mean([len(sent[0]) for sent in torch_train], dtype=int)
print(a, b)
train_maxlen = (a + b) /2
print(train_maxlen)

3160 451
1805.5


In [ ]:
l1 = [len(i[0]) for i in torch_train]
l2 = [len(i[0]) for i in torch_val]
max(max(l1),max(l2))

3160

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
max_len = 64
batch_size = 6
warmup_ratio = 0.1
num_epochs = 7
max_grad_norm = 1
log_interval = 200
learning_rate =  0.01

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
data_train = BERTDataset(torch_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(torch_val, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5, shuffle=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=20,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict=False)

        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out) 

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()


t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    
    
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        train_acc += calc_accuracy(out, label)
        # print(train_acc)
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/1167 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 train acc 0.36975149957155096


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 6 test acc 0.46899999999999986


  0%|          | 0/1167 [00:00<?, ?it/s]

epoch 7 train acc 0.46086832333618893


  0%|          | 0/500 [00:00<?, ?it/s]

epoch 7 test acc 0.46900000000000025


In [ ]:
print(model)

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [ ]:
def softmax(vals, idx):
    valscpu = vals.cpu().detach().squeeze(0)
    a = 0
    for i in valscpu:
        a += np.exp(i)
    return ((np.exp(valscpu[idx]))/a).item() * 100

def testModel(model, seq):
    cate = category
    tmp = [seq]
    transform = nlp.data.BERTSentenceTransform(tok, max_len, pad=True, pair=False)
    tokenized = transform(tmp)

    model.eval()
    result = model(torch.tensor([tokenized[0]]).to(device), [tokenized[1]], torch.tensor(tokenized[2]).to(device))
    idx = result.argmax().cpu().item()
    print("정책의 [대상특성] 태그는:", cate[idx])
    print("신뢰도는:", "{:.2f}%".format(softmax(result,idx)))

In [ ]:
testModel(model, "어쩌고 저쩌고 특성 장애를 가진 장애인이며, 5~10세 이상 장애를 가진 장애인")

정책의 [대상특성] 태그는: 해당사항없음
신뢰도는: 99.21%


# tf - 대상특성

## 1. 리스트로 넣어보기

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [ ]:
# 2
# 한글, 숫자, 영어 빼고 전부 제거
def sub_special(s):
  return re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z ~]',' ',str(s))

# 형태소 분석, 불용어 처리
def morph_and_stopword(s):
  STOP_WORDS = ['의','가','은','들','는','좀','잘','걍','과','를','으로','에','와','한','하다']
  okt = Okt()
  token_ls = []

  #형태소 분석
  tmp = okt.morphs(s, stem=True)

  #불용어 처리
  for token in tmp:
    if token not in STOP_WORDS:
      token_ls.append(token)
  return token_ls

In [ ]:
# 2
class Policy_word_change():
  # age, location 은 키워드 변환 없음

  # 대상특성 (상위태그만)
  def target(self, policy):
    df = policy.fillna('')
    change_col_1 = ['서비스명', '선정기준', '서비스목적', '지원내용', '지원대상']
    for name in change_col_1:
      df[name] = df[name].apply(sub_special).apply(morph_and_stopword)
      
      # 장애인, 난임, 제대군인, 다문화가족, 북한이탈주민, 한부모가정/조손가정, 1인가구, 다자녀가정, 독거노인, 입양가정, 외국인, 한센인, 이산가족, 기관/시설 제외
      t1 = '농업|농축|어업|원예|축산|임업|농산물|재배|귀농|영농|축산|과수|과일|유기농|화훼|원예|작물|토지|토양|농지|농가|소유|연근해|구획|어선|수산|양식|임업|사육'
      t1_ans = '농축수산인'
      t2 = '환자|입원|질병|부상|격리|난치|암|치매|요양'
      t2_ans = '질병/부상/질환자'
      t3 = '임신|임산부|출산|산모'
      t3_ans = '임신부'
      t4 = '출영유아|영아|태아|신생아|모유|분만'
      t4_ans = '출산후'
      t5 = '유공자|독립|보훈|순국선열|애국지사'
      t5_ans = '국가유공자'
      target_word = [t1, t2, t3, t4, t5]
      target_tag = [t1_ans, t2_ans, t3_ans, t4_ans, t5_ans]

      for word, tag in zip(target_word, target_tag):
        df[name] = df[name].apply(lambda x: [re.sub(word, tag, text) for text in x])
    
    return df


In [ ]:
# 2
def policy_target_model_data(policy):
  df = policy.copy()
  policy_word_change = Policy_word_change()
  df = policy_word_change.target(df)
  x = df['서비스명'] + df['선정기준'] + df['서비스목적'] + df['지원내용'] + df['지원대상']
  y = policy_y['대상특성']
  x_train, x_val, y_train, y_val = train_test_split(x, y, random_state=2022, test_size = 0.3)
  return x_train, x_val, y_train, y_val

In [ ]:
tf_train, tf_val, tf_train, tf_val = policy_target_model_data(policy_x)

In [ ]:
len(tf_train), len(tf_val), len(tf_train), len(tf_val)

(350, 150, 350, 150)

In [ ]:
for i in tf_train[:10]:
  print(i)

택시 산업 발전 지원  지 방재 정법 따르다 법률 또는 조례 통합 브랜드 콜 택시 구축 사업 등 택시 운송 사업 발전 을 위 보조금 지출 근거 마련 택시 운송 사업 발전 관 법률 따르다 택시 운송 사업 서비스 향상 을 위 보조금 지원 규정 마련 운임 요금 결제 시스템 영상 기록 시스템 등 주민 에게 편의 제공 을 위 시설 이나 장비 확충 개선 사업 택시 이용 승객 안전 을 위 교통 안전 시설 물 설치 택시 요금 카드 수수료 및 단말기 통신 료 지원 사업 택시 호출 시스템 및 첨단 교통 정보 시스템 지원 사업 그 밖에 군수 필요하다 인정 사업 자동차 교통 교통 도로 교통 택시 운송 사업자 일반
언어 발달 지원 사업 자격 확인 선정 감각 적 장애 부모 자녀 에게 필요하다 언어 발달 지원 서비스 제공 아동 건강하다 성장 지원 및 장애 가족 자체 역량 강화 바 우 처 지원 액 기초 생활 보장 수급 자 22만 원 지원 본인 부담 금 면제 차 상위 계층 20만 원 본인 부담 금 2만 원 전국 가구 평균 소득 50 이하 18만 원 본인 부담 금 4만 원 전국 가구 평균 소득 100 이하 16만 원 본인 부담 금 6만 원 서비스 내용 언어 발달 진단 언어 듣다 힘 재활 독서 지도 수화 지도 결혼 육아 교육 보육 교육 지원 보육 지원 결혼 육아 교육 보육 교육 지원 장애인 장애 부모 만 12 세 미만 비장 애인 전국 가구 평균 소득 100 이하 가구 장애 부모 한쪽 부모 및 조 손 정의 한쪽 부모 시각 청각 언어 지적 자폐 성 뇌 병변 등록 장애인 장애인 일반
보육 교사 근무 환경 개 선비 지원 어린이집 임 면 보고 되어다 반 을 맡다 있다 담임 교사 및 교사 겸직 하고 있다 원장 보육 교사 및 교사 겸직 원장 근로 여건 개선 을 위해 근무 환경 개 선비 지원 교사 근무 환경 개 선비 어린이집 에서 평일 8시간 을 원칙 근무 하고 반 을 맡다 있다 담임 교사 보육 교사 특수 교사 로 월 15일 이상 근무 사람 교사 겸직 원 장지원 비 어린이집 에서 원장 근무 사람 중 반 담임

In [ ]:
a = max(len(sent) for sent in tf_train)
b = np.mean([len(sent) for sent in tf_train], dtype=int)
print(a, b)
train_maxlen = (a + b) /2
print(train_maxlen)

3029 449
1739.0


In [ ]:
tf_train_encoded = pad_sequences([tokenizer.encode(txt) for txt in tf_train], # encode 말고 convert_tokens_to_ids?
                          maxlen=train_maxlen, dtype = 'float', truncating="post", padding="post")

TypeError: ignored

In [ ]:
# 연습
tokenizer.encode('안녕')

[101, 9521, 118741, 102]

In [ ]:
# 연습
tokenizer.convert_tokens_to_ids('안녕')

4725     [택시, 산업, 발전, 지원, 지, 방재, 정법, 따르다, 법률, 또는, 조례, 통...
37665    [언어, 발달, 지원, 사업, 자격, 확인, 선정, 감각, 적, 장애, 부모, 자녀...
28692    [보육, 교사, 근무, 환경, 개, 선비, 지원, 어린이집, 임, 면, 보고, 되어...
56366    [농, 공, 단지, 대체, 입, 주자, 대한, 감다, 산업, 입지, 및, 개발, 관...
30013    [방문, 건강, 관리, 서비스, 제공, 취약, 계층, 건강, 인식, 제, 고, 자가...
                               ...                        
25952    [지역, 공동체, 일자리, 제공, ㅇ, 만, 18, 세, 이상인, 자로, 서, 가구...
14635    [노인, 안, 검진, 및, 개다, 수술, 지원, o, 저, 소득, 층, 노인, 등,...
24661    [다문화, 가족, 보육료, 지원, 다문화, 가족, 어린이집, 이용, 영, 유아, 대...
32063    [기초, 생활, 보장, 수급, 자, 생활, 부조, 설, 추석, 명절, 을, 맞이, ...
56087    [가정, 양육, 수당, 지원, 어린이집, 이나, 유치원, 유치원, 동일하다, 성격,...
Length: 350, dtype: object

## 2. str로 넣어보기

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [ ]:
# 1
# 한글, 숫자, 영어 빼고 전부 제거
def sub_special(s):
  return re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z ~]',' ',str(s))

# 형태소 분석, 불용어 처리
def morph_and_stopword(s):
  STOP_WORDS = ['의','가','은','들','는','좀','잘','걍','과','를','으로','에','와','한','하다']
  okt = Okt()
  token_ls = []

  #형태소 분석
  tmp = okt.morphs(s, stem=True)

  #불용어 처리
  for token in tmp:
    if token not in STOP_WORDS:
      token_ls.append(token)
  return ' '.join(token_ls)

In [ ]:
# 1
class Policy_word_change():
  # age, location 은 키워드 변환 없음

  # 대상특성 (상위태그만)
  def target(self, policy):
    df = policy.fillna('')
    change_col_1 = ['서비스명', '선정기준', '서비스목적', '지원내용', '지원대상']
    for name in change_col_1:
      df[name] = df[name].apply(sub_special).apply(morph_and_stopword)
      
      # 장애인, 난임, 제대군인, 다문화가족, 북한이탈주민, 한부모가정/조손가정, 1인가구, 다자녀가정, 독거노인, 입양가정, 외국인, 한센인, 이산가족, 기관/시설 제외
      t1 = '농업|농축|어업|원예|축산|임업|농산물|재배|귀농|영농|축산|과수|과일|유기농|화훼|원예|작물|토지|토양|농지|농가|소유|연근해|구획|어선|수산|양식|임업|사육'
      t1_ans = '농축수산인'
      t2 = '환자|입원|질병|부상|격리|난치|암|치매|요양'
      t2_ans = '질병/부상/질환자'
      t3 = '임신|임산부|출산|산모'
      t3_ans = '임신부'
      t4 = '출영유아|영아|태아|신생아|모유|분만'
      t4_ans = '출산후'
      t5 = '유공자|독립|보훈|순국선열|애국지사'
      t5_ans = '국가유공자'
      target_word = [t1, t2, t3, t4, t5]
      target_tag = [t1_ans, t2_ans, t3_ans, t4_ans, t5_ans]

      for word, tag in zip(target_word, target_tag):
        df[name] = df[name].apply(lambda x: re.sub(word, tag, x))
    return df


In [ ]:
def policy_target_model_data(policy):
  df = policy.copy()
  policy_word_change = Policy_word_change()
  df = policy_word_change.target(df)
  x = df['서비스명'] + ' ' + df['선정기준'] + ' ' + df['서비스목적'] + ' ' + df['지원내용'] + ' ' + df['지원대상']
  y = policy_y['대상특성']
  x_train, x_val, y_train, y_val = train_test_split(x, y, random_state=2022, test_size = 0.3)
  return x_train, x_val, y_train, y_val

In [ ]:
x_train, x_val, y_train, y_val = policy_target_model_data(policy_x)

In [ ]:
len(x_train), len(x_val), len(y_train), len(y_val)

(350, 150, 350, 150)

In [ ]:
for i in x_train[:10]:
  print(i)

택시 산업 발전 지원  지 방재 정법 따르다 법률 또는 조례 통합 브랜드 콜 택시 구축 사업 등 택시 운송 사업 발전 을 위 보조금 지출 근거 마련 택시 운송 사업 발전 관 법률 따르다 택시 운송 사업 서비스 향상 을 위 보조금 지원 규정 마련 운임 요금 결제 시스템 영상 기록 시스템 등 주민 에게 편의 제공 을 위 시설 이나 장비 확충 개선 사업 택시 이용 승객 안전 을 위 교통 안전 시설 물 설치 택시 요금 카드 수수료 및 단말기 통신 료 지원 사업 택시 호출 시스템 및 첨단 교통 정보 시스템 지원 사업 그 밖에 군수 필요하다 인정 사업 자동차 교통 교통 도로 교통 택시 운송 사업자 일반
언어 발달 지원 사업 자격 확인 선정 감각 적 장애 부모 자녀 에게 필요하다 언어 발달 지원 서비스 제공 아동 건강하다 성장 지원 및 장애 가족 자체 역량 강화 바 우 처 지원 액 기초 생활 보장 수급 자 22만 원 지원 본인 부담 금 면제 차 상위 계층 20만 원 본인 부담 금 2만 원 전국 가구 평균 소득 50 이하 18만 원 본인 부담 금 4만 원 전국 가구 평균 소득 100 이하 16만 원 본인 부담 금 6만 원 서비스 내용 언어 발달 진단 언어 듣다 힘 재활 독서 지도 수화 지도 결혼 육아 교육 보육 교육 지원 보육 지원 결혼 육아 교육 보육 교육 지원 장애인 장애 부모 만 12 세 미만 비장 애인 전국 가구 평균 소득 100 이하 가구 장애 부모 한쪽 부모 및 조 손 정의 한쪽 부모 시각 청각 언어 지적 자폐 성 뇌 병변 등록 장애인 장애인 일반
보육 교사 근무 환경 개 선비 지원 어린이집 임 면 보고 되어다 반 을 맡다 있다 담임 교사 및 교사 겸직 하고 있다 원장 보육 교사 및 교사 겸직 원장 근로 여건 개선 을 위해 근무 환경 개 선비 지원 교사 근무 환경 개 선비 어린이집 에서 평일 8시간 을 원칙 근무 하고 반 을 맡다 있다 담임 교사 보육 교사 특수 교사 로 월 15일 이상 근무 사람 교사 겸직 원 장지원 비 어린이집 에서 원장 근무 사람 중 반 담임

In [ ]:
a = max(len(sent) for sent in x_train)
b = np.mean([len(sent) for sent in x_train], dtype=int)
print(a, b)
train_maxlen = (a + b) /2
print(train_maxlen)

3029 449
1739.0


In [ ]:
from keras.preprocessing.sequence import pad_sequences

#1
x_train_encoded = pad_sequences([tokenizer.encode(txt) for txt in x_train], # encode 말고 convert_tokens_to_ids?
                          maxlen=train_maxlen, dtype = 'float', truncating="post", padding="post")

TypeError: ignored

In [ ]:
# 연습
tokenizer.encode('안녕')

[101, 9521, 118741, 102]

In [ ]:
x_train

4725     [택시, 산업, 발전, 지원, 지, 방재, 정법, 따르다, 법률, 또는, 조례, 통...
37665    [언어, 발달, 지원, 사업, 자격, 확인, 선정, 감각, 적, 장애, 부모, 자녀...
28692    [보육, 교사, 근무, 환경, 개, 선비, 지원, 어린이집, 임, 면, 보고, 되어...
56366    [농, 공, 단지, 대체, 입, 주자, 대한, 감다, 산업, 입지, 및, 개발, 관...
30013    [방문, 건강, 관리, 서비스, 제공, 취약, 계층, 건강, 인식, 제, 고, 자가...
                               ...                        
25952    [지역, 공동체, 일자리, 제공, ㅇ, 만, 18, 세, 이상인, 자로, 서, 가구...
14635    [노인, 안, 검진, 및, 개다, 수술, 지원, o, 저, 소득, 층, 노인, 등,...
24661    [다문화, 가족, 보육료, 지원, 다문화, 가족, 어린이집, 이용, 영, 유아, 대...
32063    [기초, 생활, 보장, 수급, 자, 생활, 부조, 설, 추석, 명절, 을, 맞이, ...
56087    [가정, 양육, 수당, 지원, 어린이집, 이나, 유치원, 유치원, 동일하다, 성격,...
Length: 350, dtype: object